<h1>Image Exporter for Google Earth Engine</h1>

The code below is the first version of the image exporter. It takes a KML file that's imported from google earth pro, and transforms the polygon in the file into an image that is then exported to google drive

<h2>Imports</h2>

In [1]:
import ee
import geemap
import geopandas as gpd
from shapely.ops import transform

/Users/carrollcj/proj/golfmapper/venv/lib/python3.11/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


<h2>Authenticating and Initializing Google Earth Engine</h2>

In [3]:
# Initialize
try:
    ee.Initialize(project='golf-mapper-463720')
    print("Earth Engine initialized successfully.")
except Exception as e:
    print(f"Initialization failed: {e}")
    ee.Authenticate()
    ee.Initialize(project='golf-mapper-463720')

Earth Engine initialized successfully.


<h2>Data Cleaning, File Conversion</h2>
By default, the KML file from Earth Pro has a Z-axis, which is unable to be interpreted by Earth Engine. The code below removes the Z-axis component from the file. After the removal, the code converts the KML file into an Earth Engine Object. Lastly, the geometry is converted into.a 'Geometry' type variable, which allows it to be passed to the image clipping and export functions. 

NOTE: the image can use the 'geometry variable', but later on when we export to drive it needs to be a 'Geometry' type variable, which is called 'my_region' in this cell

In [4]:
# Read your KML
#gdf = geopandas data frame
#gdf_to_ee() is a built-in earthengine function
gdf = gpd.read_file('/Users/carrollcj/proj/golfmapper/kml_files/OS_hole1.kml')

# Function to remove Z coordinates
def remove_z_dimension(geom):
    if geom.has_z:
        return transform(lambda x, y, z=None: (x, y), geom)
    return geom

# Apply to all geometries
gdf['geometry'] = gdf.geometry.apply(remove_z_dimension)

# Verify it worked
print("After Z removal:")
print("Geometry types:", gdf.geometry.type.value_counts())
print("Sample geometry:", gdf.geometry.iloc[0])

# Now try the Earth Engine conversion
geometry = geemap.gdf_to_ee(gdf)
print("Success! Converted to Earth Engine geometry")
print(f"GEOMETRY TYPE: {type(geometry)}")
print('===============')

print('Converting geometry type...')
my_region = geometry.geometry()
print(f"CONVERTED REGION TYPE: {type(my_region)} ✅")
print(f"CONVERTED REGION INFO: {my_region.getInfo()}")
print('===============')

#We will pass the 'my_region' variable to the exporter function later on

After Z removal:
Geometry types: Polygon    1
Name: count, dtype: int64
Sample geometry: POLYGON ((-70.6113298690445 41.90304433009476, -70.60572657811036 41.90567036556823, -70.60662374388424 41.90661736167065, -70.61204505244493 41.90410616553497, -70.6113298690445 41.90304433009476))
Success! Converted to Earth Engine geometry
GEOMETRY TYPE: <class 'ee.featurecollection.FeatureCollection'>
Converting geometry type...
CONVERTED REGION TYPE: <class 'ee.geometry.Geometry'> ✅
CONVERTED REGION INFO: {'type': 'Polygon', 'coordinates': [[[-70.6113298690445, 41.90304433009476], [-70.60572657811036, 41.90567036556823], [-70.60662374388424, 41.90661736167065], [-70.61204505244493, 41.90410616553497], [-70.6113298690445, 41.90304433009476]]]}


<h2>Clipping the Satellite Image of the Region</h2>

Using images from the sentinel satellite

In [7]:
# start_date = '2023-01-01'
# end_date = '2023-12-31'

# # Get Sentinel-2 imagery
# sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
#     .filterDate(start_date, end_date) \
#     .filterBounds(geometry) \
#     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
#     .median()  # Get median composite


# rgb_image = sentinel.select(['B4', 'B3', 'B2']).multiply(0.0001)

# # Clip the image to your polygon
# clipped_image = rgb_image.clip(geometry)

#THE CODE ABOVE IS FUNCTIONAL
#Implementing a new clipping strategy (1) that will result in clearer images

print("=== SOLUTION 1: Optimized Sentinel-2 ===")

# Use more restrictive cloud filtering and recent data
start_date = '2024-01-01'
end_date = '2024-12-31'

# Get the cleanest Sentinel-2 imagery possible
sentinel_clean = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 3)) \
    .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', 3)) \
    .median()

# Select RGB bands and apply proper scaling
rgb_image = sentinel_clean.select(['B4', 'B3', 'B2']).multiply(0.0001)
clipped_image = rgb_image.clip(geometry)

# Enhanced visualization parameters for golf courses
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.02,    # Slightly higher min for better contrast
    'max': 0.28,    # Adjusted max for golf course greens
    'gamma': 1.6    # Higher gamma for more vibrant colors
}

vis_image = clipped_image.visualize(**vis_params)

# Export with optimal settings
task1 = ee.batch.Export.image.toDrive(
    image=vis_image,
    description='solution1_optimized_sentinel',
    folder='GEE_Export_Test',
    fileNamePrefix='golf_solution1_optimized',
    scale=10,  # Native resolution
    region=my_region,
    maxPixels=1e9,
    formatOptions={'cloudOptimized': True}
)

task1.start()
print(f"Solution 1 task status: {task1.status()}")


#Solution 2:


print("\n=== SOLUTION 2: Pan-Sharpening ===")

def enhanced_pan_sharpen(image):
    """
    Advanced pan-sharpening using multiple bands for better detail
    """
    # RGB bands (10m resolution)
    rgb = image.select(['B4', 'B3', 'B2'])
    
    # Use B8 (NIR) as the panchromatic band - it's also 10m but has more detail
    pan = image.select(['B8'])
    
    # Alternative: use B8A (narrow NIR) for even better results
    # pan = image.select(['B8A'])  # This is 20m, so we'll stick with B8
    
    # Calculate intensity from RGB
    intensity = rgb.reduce(ee.Reducer.mean())
    
    # Create sharpening ratio with smoothing
    ratio = pan.divide(intensity.add(0.001))  # Add small value to avoid division by zero
    
    # Apply controlled sharpening (don't oversharpen)
    sharpened = rgb.multiply(ratio).multiply(0.7).add(rgb.multiply(0.3))
    
    return sharpened

# Apply pan-sharpening
sentinel_for_sharp = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
    .map(enhanced_pan_sharpen) \
    .median()

# Scale and clip
sharp_clipped = sentinel_for_sharp.clip(geometry).multiply(0.0001)

# Visualization for sharpened image
sharp_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.015,
    'max': 0.32,
    'gamma': 1.4
}

sharp_vis_image = sharp_clipped.visualize(**sharp_vis)

# Export pan-sharpened image
task2 = ee.batch.Export.image.toDrive(
    image=sharp_vis_image,
    description='solution2_pansharpened',
    folder='GEE_Export_Test',
    fileNamePrefix='golf_solution2_pansharp',
    scale=10,
    region=my_region,
    maxPixels=1e9,
    formatOptions={'cloudOptimized': True}
)

task2.start()
print(f"Solution 2 task status: {task2.status()}")


#Solution 3:

# =============================================================================
# SOLUTION 3: LANDSAT 8/9 (Different sensor, sometimes clearer)
# =============================================================================

print("\n=== SOLUTION 3: Landsat 8/9 ===")

# Landsat Collection 2 - often has different processing that can look clearer
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2023-01-01', '2024-12-31') \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUD_COVER', 5)) \
    .median()

# Landsat scaling factors for Collection 2
# Apply scaling factors: multiply by 0.0000275 and add -0.2
landsat_rgb = landsat.select(['SR_B4', 'SR_B3', 'SR_B2']) \
    .multiply(0.0000275).add(-0.2)

landsat_clipped = landsat_rgb.clip(geometry)

# Landsat visualization (30m resolution but different characteristics)
landsat_vis = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.35,
    'gamma': 1.3
}

landsat_vis_image = landsat_clipped.visualize(**landsat_vis)

# Export Landsat image
task3 = ee.batch.Export.image.toDrive(
    image=landsat_vis_image,
    description='solution3_landsat',
    folder='GEE_Export_Test',
    fileNamePrefix='golf_solution3_landsat',
    scale=30,  # Landsat's native resolution
    region=my_region,
    maxPixels=1e9,
    formatOptions={'cloudOptimized': True}
)

task3.start()
print(f"Solution 3 task status: {task3.status()}")



=== SOLUTION 1: Optimized Sentinel-2 ===
Solution 1 task status: {'state': 'READY', 'description': 'solution1_optimized_sentinel', 'priority': 100, 'creation_timestamp_ms': 1752872952817, 'update_timestamp_ms': 1752872952817, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'LRLGTQBQAWAQWVRFQKVWRXBJ', 'name': 'projects/golf-mapper-463720/operations/LRLGTQBQAWAQWVRFQKVWRXBJ'}

=== SOLUTION 2: Pan-Sharpening ===
Solution 2 task status: {'state': 'READY', 'description': 'solution2_pansharpened', 'priority': 100, 'creation_timestamp_ms': 1752872953429, 'update_timestamp_ms': 1752872953429, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '2AYW6VIWRCFX2YBZ2IJMVPTH', 'name': 'projects/golf-mapper-463720/operations/2AYW6VIWRCFX2YBZ2IJMVPTH'}

=== SOLUTION 3: Landsat 8/9 ===
Solution 3 task status: {'state': 'READY', 'description': 'solution3_landsat', 'priority': 100, 'creation_timestamp_ms': 1752872953949, 'update_timestamp_ms': 1752872953949, 'start_timestamp_ms': 0, '

<h2>Exporting the image to Google Drive</h2>

FIXME:
- need to figure out the scale variable
- how to get an actual image not just black and white.


In [15]:
task = ee.batch.Export.image.toDrive(
    image=clipped_image,
    description='clipped_image_RGB',
    folder='GEE_Export_Test',  # Optional: creates folder in Drive
    fileNamePrefix='OS_test3',
    scale=1,  # Adjust resolution
    region=my_region,
    maxPixels=1e9,

    # formatOptions= {
    #     'cloudOptimized: True'
    # }
)
task.start()

print(f"Task status: {task.status()}")

Task status: {'state': 'READY', 'description': 'clipped_image_RGB', 'priority': 100, 'creation_timestamp_ms': 1752627058076, 'update_timestamp_ms': 1752627058076, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'UMDENY52CSMHOZAFUNEZGLFN', 'name': 'projects/golf-mapper-463720/operations/UMDENY52CSMHOZAFUNEZGLFN'}


In [5]:
Map = geemap.Map()

display(Map)

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…